# STUDY FINDINGS
The findings of the initial analysis of 15 different High Schools has been quite revealing.  The overall passing rate the aggregate of reading and math is of 80.4 percent, with a passing rate of 85.8 percent in reading and 75.0 percent for math.  
Taking a more in-depth look we saw the charter schools rise to the top, with the highest scores in math and reading, along with the highest passing rate overall.  The top 5 performing schools overall passing rate was higher than 95 percent, all of which were Charter schools.  On the contrary the lowest performing schools scored a below 74 percent overall passing rate, all of which were District schools.
Looking at average performance across all 4 years of high school (9th to 12th) of both reading and math, it is notable to point out the average across the grades and schools remained constant with marginal fluctuation.  This indicates that the students’ mean performance across their high school career did not show improvement or decline across grades.  It could be inferred that an above average performing student in 9th grade, will maintain that across his/her entire high school trajectory and vice versa.   
Analyzing the spending ranges per student, an inverse relationship between per student budget and overall passing rate was observed.   Lowest budget per student of $581 saw the highest overall passing rate of 95.0 percent. 
All margins of analysis have favored the notion of Charter schools outperforming District schools by significant percentage points. However, this analysis is not binding nor definitive.  
Charter schools have more autonomy in regard to having selective methods when it comes to admitting students.  Affluent parents also tend to invest more on tutors and extra-curricular means to provide their children a more competitive edge as they are preparing for a future college career.  District schools do not have those discriminative methods and at times are not fully funded, generating larger student to teacher ratio.  
The exploratory analysis done has yielded a great starting point for a more in-depth more descriptive analysis.  


In [7]:
import pandas as pd
import numpy as np

school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

school_data = pd.read_csv(school_data_to_load) 
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

total_schools = len(school_data_complete['School ID'].unique())
total_students = len(school_data_complete['Student ID'].unique())
total_budget = school_data_complete['budget'].unique().sum()
avg_math_score = school_data_complete['math_score'].mean()
avg_reading_score = school_data_complete['reading_score'].mean()
overall_pass_rate = (avg_math_score + avg_reading_score)/ 2
math_scr_grt_70 = (len(school_data_complete['math_score'][school_data_complete['math_score']>= 70])/total_students) * 100
reading_scr_grt_70 = (len(school_data_complete['reading_score'][school_data_complete['reading_score']>= 70])/total_students) * 100

summary_data = pd.DataFrame([
    {"Total Schools": total_schools, "Total Students": total_students, "Total Budget": total_budget,
     "Average Math Score": avg_math_score, "Average Reading Score": avg_reading_score,
     "% Passing Math": math_scr_grt_70, "% Passing Reading": reading_scr_grt_70,
     "% Overall Passing Rate": overall_pass_rate
    }])

summary_data_df = summary_data[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 
                                'Average Reading Score', '% Passing Math','% Passing Reading','% Overall Passing Rate'
                               ]]
#Formating Numbers
summary_data_df['Total Budget'] = summary_data_df['Total Budget'].map('${:,.0f}'.format)
summary_data_df['Total Students'] = summary_data_df['Total Students'].map('{:,.0f}'.format)
summary_data_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.431606


In [8]:
#Set passing scores to 1 if passed or 0 if failed
pass_70 = school_data_complete[['math_score', 'reading_score']][school_data_complete[['math_score','reading_score']] < 70].fillna(1)
pass_student = pass_70[['math_score', 'reading_score']][pass_70[['math_score', 'reading_score']] == 1].fillna(0)
nd = pd.merge(school_data_complete, pass_student, left_index=True, right_index=True)
#Set column names to be merged
columns1 = ['school_name','reading_score_x', 'math_score_x','size','budget']
columns2 = ['school_name','reading_score_y', 'math_score_y']
columns3 = ['school_name', 'type']
#Groups data after applying average and sum totals based on school name
group1 = nd[columns1].groupby('school_name').mean()
group2 = nd[columns2].groupby('school_name').sum()
group3 = nd[columns3].drop_duplicates(keep='last').set_index("school_name")
#Merge all data together to obtain the summary 
init_merge = pd.merge(group1, group2, left_index=True, right_index=True)
group_all = pd.merge(init_merge, group3, left_index=True, right_index=True)
#Create new columns to find per student budget, passing math and reading average and overall passing average
group_all["Per Student Budget"] = (group_all['budget'] / group_all["size"])
group_all['% Passing Math'] = ((group_all['math_score_y'] / group_all['size']) * 100)
group_all['% Passing Reading'] = ((group_all['reading_score_y'] / group_all['size']) * 100)
group_all["% Overall Passing Rate"] = ((group_all["% Passing Math"] + group_all["% Passing Reading"]) / 2) 

In [9]:
columns = ['type', 'math_score_x', 'reading_score_x', 'size', 'budget','Per Student Budget',
            '% Passing Math', '% Passing Reading', '% Overall Passing Rate']
new_col_map = {
    'type': 'School Type',
    'size': 'Total Students',
    'budget': 'Total School Budget',
    'Per Student Budget':'Per Student Budget',
    'math_score_x': 'Average Math Score', 
    'reading_score_x': 'Average Reading Score',
    '% Passing Math': '% Passing Math',
    '% Passing Reading': '% Passing Reading',
    '% Overall Passing Rate': '% Overall Passing Rate'
}
init_data = group_all[columns]
rename_data = init_data.rename(columns=new_col_map)
school_summary = rename_data[['School Type', 'Total Students', 'Per Student Budget', 'Average Math Score',
                  'Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate'
                  ]]
#Top Performing Schools (By Passing Rate)
school_summary.sort_values(by=['% Overall Passing Rate'], ascending=False).head()

,School Type,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,
Cabrera High School,Charter,1858.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635.0,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962.0,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283.0,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [10]:
#Bottom Performing Schools (By Passing Rate)
school_summary.sort_values(by=['% Overall Passing Rate']).head()

,School Type,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,
Rodriguez High School,District,3999.0,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [11]:
#Average math score for each grade per school
math_avg = school_data_complete.pivot_table(values='math_score', index='school_name', columns='grade')
avg_math = math_avg[['9th','10th','11th','12th']]
avg_math.columns = ['9th', '10th','11th','12th']
avg_math

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [12]:
#Average reading score for each grade per school
reading_avg = school_data_complete.pivot_table(values='reading_score', index='school_name', columns='grade')
avg_reading = reading_avg[['9th', '10th','11th','12th']]
avg_reading.columns = ['9th', '10th','11th','12th']
avg_reading

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [13]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#Scores based on spending per student analysis
bin_data = school_summary.iloc[:,2:]
bin_data['Spending Ranges (Per Student)'] =  pd.cut(school_summary['Per Student Budget'], spending_bins, labels=group_names)
spending_range = bin_data.groupby('Spending Ranges (Per Student)').mean()
#del spending_range['Per Student Budget']
spending_range

,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,,
<$585,581.000000,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,604.500000,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,635.166667,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,652.333333,76.997210,81.027843,66.164813,81.133951,73.649382


In [14]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Scores by School size
size_data = school_summary.iloc[:,3:]
size_data['School Size'] = pd.cut(school_summary['Total Students'], size_bins, labels=group_names)
size_data.groupby('School Size').mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [15]:
#Scores by school type
type_data = school_summary.groupby('School Type').mean()
del type_data['Total Students']
del type_data['Per Student Budget']
type_data

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
